In [1]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from tokenizers.processors import TemplateProcessing

c:\Users\ADMIN\Desktop\DATN\SourceCode\train_model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Đường dẫn đến thư mục chứa mô hình và processor
model_dir = "../model/kaggle/working/phoneme_recognition"

# Load processor từ thư mục
processor = Wav2Vec2Processor.from_pretrained(model_dir)

# Load mô hình từ thư mục
model = Wav2Vec2ForCTC.from_pretrained(model_dir)

# Chuyển mô hình sang chế độ đánh giá
model.eval()

# Chuyển sang GPU nếu có
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio
from typing import List, Tuple, Union

# Định nghĩa các hàm của bạn
PAD_ID = None  # Sẽ cập nhật sau khi load processor
EMPTY_ID = None

def collapse_tokens(tokens: List[Union[str, int]]) -> List[Union[str, int]]:
    prev_token = None
    out = []
    for token in tokens:
        if token != prev_token and prev_token is not None:
            out.append(prev_token)
        prev_token = token
    return out

def clean_token_ids(token_ids: List[int]) -> List[int]:
    """
    Remove [PAD] and collapse duplicated token_ids
    """
    token_ids = [x for x in token_ids if x not in [PAD_ID, EMPTY_ID]]
    token_ids = collapse_tokens(token_ids)
    return token_ids

def decode_operations(predicted_chr: str, label_chr: str, editops: List[Tuple[str, int, int]]) -> List[Tuple[str, str, str]]:
    ops = []
    for editop in editops:
        op, pred_idx, label_idx = editop
        
        if op == "insert":
            label_token = tokenizer.decode(ord(label_chr[label_idx]), group_tokens=False)
            ops.append((op, label_token, label_token))
        elif op == "delete":
            pred_token = tokenizer.decode(ord(predicted_chr[pred_idx]), group_tokens=False)
            ops.append((op, pred_token, pred_token))
        else:
            label_token = tokenizer.decode(ord(label_chr[label_idx]), group_tokens=False)
            pred_token = tokenizer.decode(ord(predicted_chr[pred_idx]), group_tokens=False)
            ops.append((op, pred_token, label_token))
            
    return ops

# Load mô hình và processor
model_dir = "../model/kaggle/working/phoneme_recognition"
processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)
tokenizer = processor.tokenizer  # Lấy tokenizer từ processor
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Cập nhật PAD_ID và EMPTY_ID
PAD_ID = tokenizer.encode("[PAD]")[0]
EMPTY_ID = tokenizer.encode(" ")[0]

# Load file âm thanh
audio_path = "../temp/test1.wav"  # Thay bằng đường dẫn file âm thanh
waveform, sample_rate = torchaudio.load(audio_path)
if sample_rate != 16000:
    resampler = torchaudio.transforms.Resample(sample_rate, 16000)
    waveform = resampler(waveform)

# Chuẩn bị đầu vào
inputs = processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
inputs = {key: value.to(device) for key, value in inputs.items()}

# Dự đoán
with torch.no_grad():
    logits = model(inputs["input_values"]).logits
    predicted_ids = torch.argmax(logits, dim=-1)[0].cpu().tolist()  # Chuyển sang list

# Làm sạch token IDs
cleaned_ids = clean_token_ids(predicted_ids)

# Giải mã sang chuỗi ký tự
predicted_chr = processor.decode(cleaned_ids)

print("Predicted transcription:", predicted_chr)

# # Nhãn thật (ground truth) - thay bằng nhãn thực tế của bạn
# label_chr = "the cat"  # Ví dụ, thay bằng nhãn thật của file âm thanh

# # Tính toán edit operations bằng thư viện Levenshtein (cần cài đặt: pip install Levenshtein)
# from Levenshtein import editops
# edit_operations = editops(predicted_chr, label_chr)

# # Phân tích các thao tác chỉnh sửa
# operations = decode_operations(predicted_chr, label_chr, edit_operations)

# # In kết quả
# print("Predicted transcription:", predicted_chr)
# print("Ground truth:", label_chr)
# print("Edit operations:", operations)

ModuleNotFoundError: No module named 'torchaudio'